# Projeto Módulo 1 
---

# Resumo

---

# Pontos Principais

---

## Pre processamento
---


## Importações de Bibliotecas

In [1]:
from matplotlib import pyplot as plt
import pandas as pd 
import seaborn as sns 

---

## O TabNet

Os dados utilizados neste projeto foram obtidos através da ferramenta TabNet do DataSUS, que esta disponível em: 

* https://datasus.saude.gov.br/informacoes-de-saude-tabnet/

<img src='https://raw.githubusercontent.com/ConradBitt/BootCamp_DataScience/master/Analise%20Dados%20Sa%C3%BAde/projeto%20modulo%201/images/TabNet.png?token=AFR43EQ7J4RD6G4PZUHPS5C7W4X3E' width=90%>

Esta é a interface da ferramenta. Nela estão disponíveis vários dados, inclusive dados que não pertencem diretamente ao DataSUS, como dados Demograficos do IBGE. Os dados que vou utilizar são:

1. AIH aprovadas por Ano/Mês processamento segundo Unidade da Federação.

2. Taxa mortalidade por Ano/Mês processamento segundo Unidade da Federação

3. Valor total por Ano/Mês processamento segundo Unidade da Federação

4. Domicílios por Abastecimento de água, Coleta de lixo e Instalações Sanitárias segundo Unidade da Federação.

5. Media Moradores por domicilios segundo Unidade da Federação

6. Populacao residente segundo Unidade da Federação
    
    **Nota sobre itens 1, 2 e 3:** A partir do processamento de junho de 2012, houve mudança na classificação da natureza e esfera dos estabelecimentos. Com isso, temos que:
      
    1. Até maio de 2012 estas informações estão disponíveis como "Natureza" e "Esfera Administrativa".

    2. De junho de 2012 a outubro de 2015, estão disponíveis tanto como "Natureza" e "Esfera Administrativa", como "Natureza Jurídica" e "Esfera Jurídica".
      
    3. A partir de novembro de 2015, estão disponíveis como "Natureza Jurídica" e "Esfera Jurídica".
    

### Fontes 

* Ministério da Saúde - Sistema de Informações Hospitalares do SUS (SIH/SUS)

* IBGE - Censos Demográfico 2010

Estes dados podem ser obtidos nas seguintes caregorias sublinhadas em vermelho no TabNet: 

<img src='https://raw.githubusercontent.com/ConradBitt/BootCamp_DataScience/master/Analise%20Dados%20Sa%C3%BAde/projeto%20modulo%201/images/Dados_disponiveis_tabnet.png?token=AFR43EX6J5E6HOLDKTRIOTC7W4YAW' width=90%>

## Importações dos Dados

Vou importar os dados que estão dentro da pasta `./dados_datasus/`:

In [2]:
AIH_aprovadas = pd.read_csv('./dados_datasus/AIH aprovadas por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=4,skipfooter=12)
taxa_mortalidade = pd.read_csv('./dados_datasus/Taxa mortalidade por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', decimal=',', skiprows=4,skipfooter=12)
valor_total = pd.read_csv('./dados_datasus/Valor total por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', decimal=',', skiprows=4,skipfooter=12)
internacoes = pd.read_csv('./dados_datasus/Internacoes por AnoMês processamento segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=12)
instalacoes_sanitarias_domicilio = pd.read_csv('./dados_datasus/Domicilios por Instalacoes sanitarias segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=2)
abastecimento_agua_domicilio = pd.read_csv('./dados_datasus/Domicílios por Abastecimento de água segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=2)
coleta_lixo_domicilio = pd.read_csv('./dados_datasus/Domicílios por Coleta de lixo segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=2)
media_moradores_domicilio = pd.read_csv('./dados_datasus/Media MoradoresDomicilios segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python',decimal=',', skiprows=3,skipfooter=8)
populacao_residente = pd.read_csv('./dados_datasus/Populacao residente segundo Unidade da Federacao.csv', encoding='ISO-8859-1', sep=';', engine='python', skiprows=3,skipfooter=17)

---
## Tratando os dados

Como o dataset `populacao_residente` do IBGE não tem os números nos índices das macrorregiões, vou adicionar esta series em todos os dataframes no lugar da coluna `Unidade da Federação`, isso vai ser feito criando um dicionário para automatizar a reatribuição das colunas.

Alguns desses dataframes tem séries temporais (`AIH_aprovadas`,`taxa_mortalidade`, `valor_total` e `internacoes`) onde uma pequena parte dos registros estão no tipo `object`, mas são numéricos, então vou transforma-los.

Outros dataframes que armazenam os dados de abastecimento de agua, instalações sanitarias e coleta de lixo são divididas em varios grupos. O tratamento que será feito é dividir cada dataframe em duas categorias: `Rede geral` e `Rede alternativa`.

Vou considerar uma `Rede Geral` os domicílios que tem abastecimento de água por uma rede de canalização, instalação sanitaria por meio de uma rede geral de esgoto ou pluvial e coleta de lixo feita por caçamba ou serviço de limpeza.

Já os domicílios que tem abastecimento de água por meio de poço, nascente, carro-pipa, cisterna, rio, açude, lago ou igarapé. Que usam instalações sanitarias como fossa séptica ou rudimendar, vala, rio, lago ou mar ou outro escoadouro, inclusive quem não tem instalação sanitária. E que descarta o lixo através da queima ou enterrando na propriedade, em terreno baldio ou logradouro, jogado em rio, lago, mar, ou outro destino, como a categoria `Rede Alternativa`.

In [3]:
Unidades_Federacao = populacao_residente['Unidade da Federação']

datasets = {'AIH_aprovadas': AIH_aprovadas,'taxa_mortalidade': taxa_mortalidade,'valor_total': valor_total,'internacoes': internacoes,'abastecimento_agua_domicilio' : abastecimento_agua_domicilio,'instalacoes_sanitarias_domicilio' : instalacoes_sanitarias_domicilio,'coleta_lixo_domicilio' : coleta_lixo_domicilio,'media_moradores_domicilio': media_moradores_domicilio,'populacao_residente': populacao_residente}

for dataset, conteudo in datasets.items():    
    #  Substituindo index
    conteudo['Unidade da Federação'] = Unidades_Federacao
    
    #  Substituindo registros com hífen
    datasets[dataset] = conteudo.replace('-', 'NaN')
    
    #  Referenciando novo conteudo.
    datasets[dataset] = conteudo.set_index('Unidade da Federação')
    
#  Reatribuindo referência as variáveis SIH
AIH_aprovadas = datasets['AIH_aprovadas'].replace('-','NaN').astype(float).fillna(method='ffill').astype(int)
taxa_mortalidade = datasets['taxa_mortalidade'].replace('-','NaN').replace('...','NaN').astype(float).fillna(method='ffill')
valor_total = datasets['valor_total'].replace('-','NaN').astype(float).fillna(method='ffill')
internacoes = datasets['internacoes'].replace('-','NaN').astype(float).fillna(method='ffill').astype(int)

#  Reatribuindo referência as variáveis
instalacoes_sanitarias_domicilio = datasets['instalacoes_sanitarias_domicilio'].replace('-','NaN').astype(float).fillna(0)
abastecimento_agua_domicilio = datasets['abastecimento_agua_domicilio'].replace('-','NaN').astype(float).fillna(0)
coleta_lixo_domicilio = datasets['coleta_lixo_domicilio'].replace('-','NaN').astype(float).fillna(0).astype(int)

#  Instalações sanitarias
instalacoes_sanitarias_domicilio['Rede Geral'] = instalacoes_sanitarias_domicilio[instalacoes_sanitarias_domicilio.columns[0]]
instalacoes_sanitarias_domicilio['Rede Alternativa'] = instalacoes_sanitarias_domicilio[instalacoes_sanitarias_domicilio.columns[1:7]].sum(axis=1)
instalacoes_sanitarias_domicilio = instalacoes_sanitarias_domicilio[['Rede Geral', 'Rede Alternativa']]

#  Abastecimento de agua em duas categorias
abastecimento_agua_domicilio['Rede Geral'] = abastecimento_agua_domicilio[abastecimento_agua_domicilio.columns[0]]
abastecimento_agua_domicilio['Rede Alternativa'] = abastecimento_agua_domicilio[abastecimento_agua_domicilio.columns[1:10]].sum(axis=1)
abastecimento_agua_domicilio = abastecimento_agua_domicilio[['Rede Geral', 'Rede Alternativa']]

#  Coleta de Lixo 
coleta_lixo_domicilio['Rede Geral'] = coleta_lixo_domicilio[coleta_lixo_domicilio.columns[:2]].sum(axis=1)
coleta_lixo_domicilio['Rede Alternativa'] = coleta_lixo_domicilio[coleta_lixo_domicilio.columns[2:-2]].sum(axis=1)
coleta_lixo_domicilio = coleta_lixo_domicilio[['Rede Geral','Rede Alternativa']]    

# Dados populacionais do IBGE
media_moradores_domicilio = datasets['media_moradores_domicilio'][['Média_moradores/domicílios']]
populacao_residente = datasets['populacao_residente'][['População_residente']]

datasets = {'AIH_aprovadas': AIH_aprovadas,'taxa_mortalidade': taxa_mortalidade,'valor_total': valor_total,'internacoes': internacoes,'abastecimento_agua_domicilio' : abastecimento_agua_domicilio,'instalacoes_sanitarias_domicilio' : instalacoes_sanitarias_domicilio,'coleta_lixo_domicilio' : coleta_lixo_domicilio,'media_moradores_domicilio': media_moradores_domicilio,'populacao_residente': populacao_residente}

for dataset, conteudo in datasets.items():    
#  Exibindo informações
    print(50*'-')
    print(5*'=', f'> Dataset: {dataset} <', 5*'=')
    print(50*'-')
    display(conteudo.sample(5))
    conteudo.info()
    print('\n')

--------------------------------------------------
===== > Dataset: AIH_aprovadas < =====
--------------------------------------------------


,2008/Jan,2008/Fev,2008/Mar,2008/Abr,2008/Mai,2008/Jun,2008/Jul,2008/Ago,2008/Set,2008/Out,...,2019/Dez,2020/Jan,2020/Fev,2020/Mar,2020/Abr,2020/Mai,2020/Jun,2020/Jul,2020/Ago,Total
Unidade da Federação,,,,,,,,,,,,,,,,,,,,,
Pará,32625,34607,38148,37876,41438,40573,42385,40874,41430,41000,...,35730,35599,36532,34955,28113,27410,29664,27215,27872,5645282
Roraima,1157,1430,1338,1442,1537,1505,1719,1687,1418,1868,...,3116,3341,3259,3119,2848,2685,2630,2565,2889,394728
Alagoas,8012,9518,10489,12479,12666,12865,12985,13169,12855,13297,...,10718,11311,10544,10882,10065,8884,8246,9694,10557,1788282
Minas Gerais,56796,76614,82076,80630,82213,90776,83298,82979,82677,83909,...,85196,91011,84923,82964,75755,77741,75997,82332,80936,12600919
Goiás,23292,31385,29412,30734,30463,25704,28013,28138,29265,27566,...,23421,23798,22688,23444,20749,20721,19440,20076,20643,3857727


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Columns: 153 entries, 2008/Jan to Total
dtypes: int64(153)
memory usage: 32.5+ KB


--------------------------------------------------
===== > Dataset: taxa_mortalidade < =====
--------------------------------------------------


,2008/Jan,2008/Fev,2008/Mar,2008/Abr,2008/Mai,2008/Jun,2008/Jul,2008/Ago,2008/Set,2008/Out,...,2019/Dez,2020/Jan,2020/Fev,2020/Mar,2020/Abr,2020/Mai,2020/Jun,2020/Jul,2020/Ago,Total
Unidade da Federação,,,,,,,,,,,,,,,,,,,,,
Distrito Federal,4.81,3.29,2.73,3.20,2.39,2.70,3.00,2.76,3.23,3.29,...,3.08,3.59,3.06,3.28,3.37,3.24,4.53,5.44,6.18,3.34
Piauí,1.56,1.94,2.00,1.54,2.13,2.10,2.08,1.77,2.08,1.99,...,3.49,3.53,3.26,3.29,3.67,3.97,4.17,6.11,6.27,2.88
Rio Grande do Norte,2.32,1.97,2.63,2.68,2.67,3.12,2.63,2.89,2.96,3.09,...,4.40,4.94,4.64,4.50,4.77,5.18,5.99,7.01,7.05,4.14
Paraná,2.72,3.69,3.36,3.47,3.59,3.61,3.94,4.09,3.65,3.86,...,4.64,4.72,4.86,4.94,5.00,5.06,5.69,6.13,7.03,4.43
Amazonas,2.16,2.04,2.18,2.34,1.91,1.92,2.10,2.09,2.00,1.96,...,3.28,3.20,3.86,3.26,4.79,10.88,10.12,6.18,4.52,2.77


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Columns: 153 entries, 2008/Jan to Total
dtypes: float64(153)
memory usage: 32.5+ KB


--------------------------------------------------
===== > Dataset: valor_total < =====
--------------------------------------------------


...

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Rede Geral        27 non-null     int64
 1   Rede Alternativa  27 non-null     int64
dtypes: int64(2)
memory usage: 648.0+ bytes


--------------------------------------------------
===== > Dataset: media_moradores_domicilio < =====
--------------------------------------------------


,Média_moradores/domicílios
Unidade da Federação,
Goiás,3.0
Acre,3.6
Roraima,3.7
Amazonas,3.9
Rondônia,3.2


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Data columns (total 1 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Média_moradores/domicílios  27 non-null     float64
dtypes: float64(1)
memory usage: 432.0+ bytes


--------------------------------------------------
===== > Dataset: populacao_residente < =====
--------------------------------------------------


,População_residente
Unidade da Federação,
Pará,7367487
Pernambuco,8745133
Amapá,625650
Rondônia,1518809
Santa Catarina,6091251


<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Rondônia to Distrito Federal
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   População_residente  27 non-null     int64
dtypes: int64(1)
memory usage: 432.0+ bytes




# Análise dos Dados
---